# Prisoner's Dilemma Game

Two agents play against each other for 10 time steps.

## Payoff Matrix
|   | C | D |
|---|---|---|
| **C** | -1, -1 | -5, 0 |
| **D** | 0, -5 | -3, -3 |

## Agent Models
- **Coop**: Always cooperates
- **Grim**: Cooperates initially, but if opponent ever defects, defects forever

In [1]:
from enum import Enum
from abc import ABC, abstractmethod

# Actions
class Action(Enum):
    COOPERATE = 'C'
    DEFECT = 'D'

# Payoff matrix: (row_player_payoff, col_player_payoff)
# Row player action is first, column player action is second
PAYOFF_MATRIX = {
    (Action.COOPERATE, Action.COOPERATE): (-1, -1),
    (Action.COOPERATE, Action.DEFECT): (-5, 0),
    (Action.DEFECT, Action.COOPERATE): (0, -5),
    (Action.DEFECT, Action.DEFECT): (-3, -3),
}

# Base Agent class
class Agent(ABC):
    def __init__(self, name: str):
        self.name = name
        self.score = 0
        self.history = []  # History of own actions
        self.opponent_history = []  # History of opponent's actions
    
    @abstractmethod
    def choose_action(self) -> Action:
        """Choose an action based on current state."""
        pass
    
    def observe(self, own_action: Action, opponent_action: Action):
        """Record the actions taken this round."""
        self.history.append(own_action)
        self.opponent_history.append(opponent_action)
    
    def receive_payoff(self, payoff: int):
        """Add payoff to total score."""
        self.score += payoff
    
    def reset(self):
        """Reset agent state for a new game."""
        self.score = 0
        self.history = []
        self.opponent_history = []

# Coop Agent: Always cooperates
class CoopAgent(Agent):
    def __init__(self, name: str = "Coop"):
        super().__init__(name)
    
    def choose_action(self) -> Action:
        return Action.COOPERATE

# Grim Agent: Cooperates until opponent defects, then defects forever
class GrimAgent(Agent):
    def __init__(self, name: str = "Grim"):
        super().__init__(name)
        self.triggered = False  # Has opponent ever defected?
    
    def choose_action(self) -> Action:
        if self.triggered:
            return Action.DEFECT
        return Action.COOPERATE
    
    def observe(self, own_action: Action, opponent_action: Action):
        super().observe(own_action, opponent_action)
        if opponent_action == Action.DEFECT:
            self.triggered = True
    
    def reset(self):
        super().reset()
        self.triggered = False

print("Agent classes defined successfully!")

Agent classes defined successfully!


In [2]:
def play_game(agent1: Agent, agent2: Agent, num_rounds: int = 10, verbose: bool = True):
    """Play the Prisoner's Dilemma game between two agents."""
    
    agent1.reset()
    agent2.reset()
    
    if verbose:
        print(f"{'='*60}")
        print(f"Game: {agent1.name} vs {agent2.name} ({num_rounds} rounds)")
        print(f"{'='*60}")
        print(f"{'Round':<8} {agent1.name:<12} {agent2.name:<12} {'Payoffs':<15}")
        print(f"{'-'*60}")
    
    for round_num in range(1, num_rounds + 1):
        # Both agents choose their actions simultaneously
        action1 = agent1.choose_action()
        action2 = agent2.choose_action()
        
        # Get payoffs from matrix
        payoff1, payoff2 = PAYOFF_MATRIX[(action1, action2)]
        
        # Update agents
        agent1.observe(action1, action2)
        agent2.observe(action2, action1)
        agent1.receive_payoff(payoff1)
        agent2.receive_payoff(payoff2)
        
        if verbose:
            print(f"{round_num:<8} {action1.value:<12} {action2.value:<12} ({payoff1:+d}, {payoff2:+d})")
    
    if verbose:
        print(f"{'-'*60}")
        print(f"Final Scores: {agent1.name} = {agent1.score}, {agent2.name} = {agent2.score}")
        print(f"{'='*60}")
    
    return agent1.score, agent2.score

print("Game function defined!")

Game function defined!


## Game 1: Coop vs Coop
Both agents always cooperate - they achieve mutual cooperation every round.

In [3]:
# Game 1: Coop vs Coop
coop1 = CoopAgent("Coop-1")
coop2 = CoopAgent("Coop-2")
play_game(coop1, coop2, num_rounds=10)

Game: Coop-1 vs Coop-2 (10 rounds)
Round    Coop-1       Coop-2       Payoffs        
------------------------------------------------------------
1        C            C            (-1, -1)
2        C            C            (-1, -1)
3        C            C            (-1, -1)
4        C            C            (-1, -1)
5        C            C            (-1, -1)
6        C            C            (-1, -1)
7        C            C            (-1, -1)
8        C            C            (-1, -1)
9        C            C            (-1, -1)
10       C            C            (-1, -1)
------------------------------------------------------------
Final Scores: Coop-1 = -10, Coop-2 = -10


(-10, -10)

## Game 2: Grim vs Grim
Both Grim agents start by cooperating. Since neither defects first, they cooperate forever.

In [4]:
# Game 2: Grim vs Grim
grim1 = GrimAgent("Grim-1")
grim2 = GrimAgent("Grim-2")
play_game(grim1, grim2, num_rounds=10)

Game: Grim-1 vs Grim-2 (10 rounds)
Round    Grim-1       Grim-2       Payoffs        
------------------------------------------------------------
1        C            C            (-1, -1)
2        C            C            (-1, -1)
3        C            C            (-1, -1)
4        C            C            (-1, -1)
5        C            C            (-1, -1)
6        C            C            (-1, -1)
7        C            C            (-1, -1)
8        C            C            (-1, -1)
9        C            C            (-1, -1)
10       C            C            (-1, -1)
------------------------------------------------------------
Final Scores: Grim-1 = -10, Grim-2 = -10


(-10, -10)

## Game 3: Coop vs Grim
Coop always cooperates, Grim also cooperates (since Coop never defects). Mutual cooperation!

In [5]:
# Game 3: Coop vs Grim
coop = CoopAgent("Coop")
grim = GrimAgent("Grim")
play_game(coop, grim, num_rounds=10)

Game: Coop vs Grim (10 rounds)
Round    Coop         Grim         Payoffs        
------------------------------------------------------------
1        C            C            (-1, -1)
2        C            C            (-1, -1)
3        C            C            (-1, -1)
4        C            C            (-1, -1)
5        C            C            (-1, -1)
6        C            C            (-1, -1)
7        C            C            (-1, -1)
8        C            C            (-1, -1)
9        C            C            (-1, -1)
10       C            C            (-1, -1)
------------------------------------------------------------
Final Scores: Coop = -10, Grim = -10


(-10, -10)

## Bonus: Adding a Defector Agent
To show how Grim reacts to defection, let's add an agent that always defects.

In [6]:
# Defector Agent: Always defects
class DefectorAgent(Agent):
    def __init__(self, name: str = "Defector"):
        super().__init__(name)
    
    def choose_action(self) -> Action:
        return Action.DEFECT

# Game 4: Grim vs Defector
# Grim cooperates on round 1, then sees defection and defects forever
grim = GrimAgent("Grim")
defector = DefectorAgent("Defector")
play_game(grim, defector, num_rounds=10)

Game: Grim vs Defector (10 rounds)
Round    Grim         Defector     Payoffs        
------------------------------------------------------------
1        C            D            (-5, +0)
2        D            D            (-3, -3)
3        D            D            (-3, -3)
4        D            D            (-3, -3)
5        D            D            (-3, -3)
6        D            D            (-3, -3)
7        D            D            (-3, -3)
8        D            D            (-3, -3)
9        D            D            (-3, -3)
10       D            D            (-3, -3)
------------------------------------------------------------
Final Scores: Grim = -32, Defector = -27


(-32, -27)

## Summary: Tournament Results
Let's run a round-robin tournament between all agent types.

In [7]:
# Round-robin tournament
agent_classes = [
    ("Coop", CoopAgent),
    ("Grim", GrimAgent),
    ("Defector", DefectorAgent)
]

print("TOURNAMENT RESULTS (10 rounds each)")
print("="*50)
print(f"{'Matchup':<25} {'Score 1':<10} {'Score 2':<10}")
print("-"*50)

for i, (name1, cls1) in enumerate(agent_classes):
    for j, (name2, cls2) in enumerate(agent_classes):
        if i <= j:  # Only play each matchup once
            agent1 = cls1(name1)
            agent2 = cls2(name2)
            score1, score2 = play_game(agent1, agent2, num_rounds=10, verbose=False)
            print(f"{name1} vs {name2:<15} {score1:<10} {score2:<10}")

print("="*50)

TOURNAMENT RESULTS (10 rounds each)
Matchup                   Score 1    Score 2   
--------------------------------------------------
Coop vs Coop            -10        -10       
Coop vs Grim            -10        -10       
Coop vs Defector        -50        0         
Grim vs Grim            -10        -10       
Grim vs Defector        -32        -27       
Defector vs Defector        -30        -30       


Key observations from the results:

Coop vs Defector: Coop gets exploited badly (-50 vs 0)
Grim vs Defector: Grim only gets exploited once, then retaliates (-32 vs -27)
Grim's "trigger" strategy protects it from ongoing exploitation

In [12]:
# Strategic Defector: cooperates until round N, then defects forever
class StrategicDefectorAgent(Agent):
    def __init__(self, name: str, defect_round: int):
        super().__init__(name)
        self.defect_round = defect_round
        self.current_round = 0
    
    def choose_action(self) -> Action:
        self.current_round += 1
        if self.current_round >= self.defect_round:
            return Action.DEFECT
        return Action.COOPERATE
    
    def reset(self):
        super().reset()
        self.current_round = 0

def run_multi_agent_game(agents, num_rounds=10, verbose=False):
    """Run a multi-agent game and return final scores."""
    for agent in agents:
        agent.reset()
    
    cumulative_scores = {agent.name: 0 for agent in agents}
    
    for round_num in range(1, num_rounds + 1):
        actions = {agent.name: agent.choose_action() for agent in agents}
        round_payoffs = {agent.name: 0 for agent in agents}
        
        for i, agent1 in enumerate(agents):
            for j, agent2 in enumerate(agents):
                if i < j:
                    action1 = actions[agent1.name]
                    action2 = actions[agent2.name]
                    payoff1, payoff2 = PAYOFF_MATRIX[(action1, action2)]
                    round_payoffs[agent1.name] += payoff1
                    round_payoffs[agent2.name] += payoff2
        
        for agent in agents:
            cumulative_scores[agent.name] += round_payoffs[agent.name]
        
        for agent in agents:
            own_action = actions[agent.name]
            for other in agents:
                if other.name != agent.name:
                    agent.observe(own_action, actions[other.name])
        
        if verbose:
            action_str = "  ".join([f"{a.name}: {actions[a.name].value}" for a in agents])
            payoff_str = "  ".join([f"{a.name}: {round_payoffs[a.name]:+d}" for a in agents])
            print(f"Round {round_num:2d} | Actions: {action_str}")
            print(f"         | Payoffs: {payoff_str}")
            print("-"*80)
    
    return cumulative_scores

# Analyze optimal defection round
print("ANALYSIS: When should the Strategic Defector start defecting?")
print("="*80)
print(f"{'Defect Round':<15} {'Coop-1':<10} {'Coop-2':<10} {'Grim':<10} {'Defector':<10}")
print("-"*80)

results = []
for defect_round in range(1, 12):  # 1-10 means defect from that round, 11 means never
    agents = [
        CoopAgent("Coop-1"),
        CoopAgent("Coop-2"),
        GrimAgent("Grim"),
        StrategicDefectorAgent("Defector", defect_round)
    ]
    scores = run_multi_agent_game(agents, num_rounds=10)
    results.append((defect_round, scores))
    
    label = f"Round {defect_round}" if defect_round <= 10 else "Never"
    print(f"{label:<15} {scores['Coop-1']:<10} {scores['Coop-2']:<10} {scores['Grim']:<10} {scores['Defector']:<10}")

# Find optimal
optimal = max(results, key=lambda x: x[1]['Defector'])
print("="*80)
print(f"\nOPTIMAL: Defect starting from Round {optimal[0]} → Defector score: {optimal[1]['Defector']}")

# Show detailed game with optimal strategy
print("\n" + "="*80)
print(f"DETAILED GAME: Defector starts defecting at Round {optimal[0]}")
print("="*80)

agents = [
    CoopAgent("Coop-1"),
    CoopAgent("Coop-2"),
    GrimAgent("Grim"),
    StrategicDefectorAgent("Defector", optimal[0])
]
final_scores = run_multi_agent_game(agents, num_rounds=10, verbose=True)

print("="*80)
print("FINAL SCORES")
print("-"*80)
for name, score in sorted(final_scores.items(), key=lambda x: x[1], reverse=True):
    print(f"{name:<15} {score:>5}")
print("="*80)

ANALYSIS: When should the Strategic Defector start defecting?
Defect Round    Coop-1     Coop-2     Grim       Defector  
--------------------------------------------------------------------------------
Round 1         -106       -106       -34        -27       
Round 2         -98        -98        -34        -27       
Round 3         -90        -90        -34        -27       
Round 4         -82        -82        -34        -27       
Round 5         -74        -74        -34        -27       
Round 6         -66        -66        -34        -27       
Round 7         -58        -58        -34        -27       
Round 8         -50        -50        -34        -27       
Round 9         -42        -42        -34        -27       
Round 10        -34        -34        -34        -27       
Never           -30        -30        -30        -30       

OPTIMAL: Defect starting from Round 1 → Defector score: -27

DETAILED GAME: Defector starts defecting at Round 1
Round  1 | Actions: Coo

In [14]:
# BAYESIAN PRISONER'S DILEMMA
# Agent doesn't know opponent types, must infer from observed actions

import random

# Possible opponent types and their action probabilities
# P(action=C | type) for each type
TYPE_ACTION_PROBS = {
    'Coop': {Action.COOPERATE: 1.0, Action.DEFECT: 0.0},
    'Defector': {Action.COOPERATE: 0.0, Action.DEFECT: 1.0},
    'Grim': {Action.COOPERATE: 0.9, Action.DEFECT: 0.1},  # Grim mostly cooperates unless triggered
}

class BayesianAgent(Agent):
    """
    A Bayesian agent that:
    1. Maintains beliefs (probability distribution) over opponent types
    2. Updates beliefs using Bayes' rule after observing actions
    3. Chooses actions to maximize expected utility given beliefs
    """
    def __init__(self, name: str = "Bayesian"):
        super().__init__(name)
        # Prior beliefs about each opponent: P(type)
        # Key: opponent_name, Value: dict of type -> probability
        self.beliefs = {}
        self.opponent_defected = {}  # Track if opponent ever defected (for Grim likelihood)
    
    def initialize_beliefs(self, opponent_names):
        """Set uniform prior over opponent types."""
        for opp in opponent_names:
            self.beliefs[opp] = {'Coop': 1/3, 'Defector': 1/3, 'Grim': 1/3}
            self.opponent_defected[opp] = False
    
    def update_beliefs(self, opponent_name: str, observed_action: Action):
        """Update beliefs about opponent using Bayes' rule."""
        if opponent_name not in self.beliefs:
            return
        
        prior = self.beliefs[opponent_name]
        
        # Compute likelihood P(action | type) for each type
        likelihoods = {}
        for type_name in prior:
            if type_name == 'Grim':
                # Grim's behavior depends on whether WE defected
                if self.opponent_defected.get(opponent_name, False):
                    # If Grim was triggered, they defect with high prob
                    likelihoods[type_name] = 0.95 if observed_action == Action.DEFECT else 0.05
                else:
                    # Grim cooperates until triggered
                    likelihoods[type_name] = 0.95 if observed_action == Action.COOPERATE else 0.05
            else:
                likelihoods[type_name] = TYPE_ACTION_PROBS[type_name][observed_action]
                # Add small epsilon to avoid zero probabilities
                likelihoods[type_name] = max(likelihoods[type_name], 0.01)
        
        # Bayes' rule: P(type | action) = P(action | type) * P(type) / P(action)
        unnormalized = {t: likelihoods[t] * prior[t] for t in prior}
        total = sum(unnormalized.values())
        
        if total > 0:
            self.beliefs[opponent_name] = {t: p / total for t, p in unnormalized.items()}
    
    def expected_payoff(self, my_action: Action) -> float:
        """Calculate expected payoff for an action given current beliefs."""
        total_expected = 0
        
        for opp_name, type_probs in self.beliefs.items():
            for type_name, prob in type_probs.items():
                # Expected action from this type
                if type_name == 'Coop':
                    opp_action = Action.COOPERATE
                elif type_name == 'Defector':
                    opp_action = Action.DEFECT
                else:  # Grim
                    if self.opponent_defected.get(opp_name, False):
                        opp_action = Action.DEFECT
                    else:
                        opp_action = Action.COOPERATE
                
                payoff, _ = PAYOFF_MATRIX[(my_action, opp_action)]
                total_expected += prob * payoff
        
        return total_expected
    
    def choose_action(self) -> Action:
        """Choose action that maximizes expected utility."""
        exp_coop = self.expected_payoff(Action.COOPERATE)
        exp_defect = self.expected_payoff(Action.DEFECT)
        
        # Store for display
        self.last_expected = {'C': exp_coop, 'D': exp_defect}
        
        return Action.COOPERATE if exp_coop >= exp_defect else Action.DEFECT
    
    def observe_opponent_action(self, opponent_name: str, opponent_action: Action, my_action: Action):
        """Observe and update beliefs about a specific opponent."""
        # Track if we defected (which would trigger Grim)
        if my_action == Action.DEFECT:
            self.opponent_defected[opponent_name] = True
        self.update_beliefs(opponent_name, opponent_action)
    
    def reset(self):
        super().reset()
        self.beliefs = {}
        self.opponent_defected = {}

def run_bayesian_game(agents, num_rounds=10, bayesian_agent_name="Bayesian"):
    """Run game with Bayesian agent tracking beliefs."""
    for agent in agents:
        agent.reset()
    
    # Initialize Bayesian agent's beliefs
    bayesian = next((a for a in agents if a.name == bayesian_agent_name), None)
    if bayesian:
        opponent_names = [a.name for a in agents if a.name != bayesian_agent_name]
        bayesian.initialize_beliefs(opponent_names)
    
    cumulative_scores = {agent.name: 0 for agent in agents}
    
    print(f"{'='*90}")
    print("BAYESIAN PRISONER'S DILEMMA")
    print("The Bayesian agent doesn't know opponent types - it must infer from observed actions")
    print(f"{'='*90}\n")
    
    for round_num in range(1, num_rounds + 1):
        print(f"ROUND {round_num}")
        print("-"*90)
        
        # Show Bayesian agent's beliefs before action
        if bayesian:
            print("Bayesian's beliefs about opponent types:")
            for opp, probs in bayesian.beliefs.items():
                prob_str = ", ".join([f"{t}: {p:.2%}" for t, p in probs.items()])
                print(f"  {opp}: [{prob_str}]")
        
        actions = {agent.name: agent.choose_action() for agent in agents}
        
        # Show expected utilities
        if bayesian and hasattr(bayesian, 'last_expected'):
            print(f"Expected payoffs: Cooperate={bayesian.last_expected['C']:.2f}, Defect={bayesian.last_expected['D']:.2f}")
        
        round_payoffs = {agent.name: 0 for agent in agents}
        
        for i, agent1 in enumerate(agents):
            for j, agent2 in enumerate(agents):
                if i < j:
                    action1 = actions[agent1.name]
                    action2 = actions[agent2.name]
                    payoff1, payoff2 = PAYOFF_MATRIX[(action1, action2)]
                    round_payoffs[agent1.name] += payoff1
                    round_payoffs[agent2.name] += payoff2
        
        for agent in agents:
            cumulative_scores[agent.name] += round_payoffs[agent.name]
        
        # Update Bayesian agent's beliefs based on observed actions
        if bayesian:
            my_action = actions[bayesian.name]
            for other in agents:
                if other.name != bayesian.name:
                    bayesian.observe_opponent_action(other.name, actions[other.name], my_action)
        
        # Regular observation for other agents
        for agent in agents:
            own_action = actions[agent.name]
            for other in agents:
                if other.name != agent.name:
                    agent.observe(own_action, actions[other.name])
        
        action_str = "  ".join([f"{a.name}: {actions[a.name].value}" for a in agents])
        payoff_str = "  ".join([f"{a.name}: {round_payoffs[a.name]:+d}" for a in agents])
        print(f"Actions: {action_str}")
        print(f"Payoffs: {payoff_str}")
        print()
    
    return cumulative_scores

# Run Bayesian game: Bayesian agent vs unknown opponents (Coop, Grim, Defector)
print("SETUP: Bayesian agent plays against 3 opponents whose types it doesn't know")
print("True types: Coop, Grim, Defector\n")

agents = [
    BayesianAgent("Bayesian"),
    CoopAgent("Player-A"),      # Actually Coop
    GrimAgent("Player-B"),      # Actually Grim  
    DefectorAgent("Player-C")   # Actually Defector
]

final_scores = run_bayesian_game(agents, num_rounds=10, bayesian_agent_name="Bayesian")

print("="*90)
print("FINAL SCORES")
print("-"*90)
for name, score in sorted(final_scores.items(), key=lambda x: x[1], reverse=True):
    agent = next(a for a in agents if a.name == name)
    true_type = type(agent).__name__.replace("Agent", "")
    print(f"{name:<15} (True type: {true_type:<10}) Score: {score:>5}")
print("="*90)

# Show final beliefs
bayesian = agents[0]
print("\nBayesian's FINAL beliefs about opponent types:")
print("-"*90)
for opp, probs in bayesian.beliefs.items():
    agent = next(a for a in agents if a.name == opp)
    true_type = type(agent).__name__.replace("Agent", "")
    prob_str = ", ".join([f"{t}: {p:.2%}" for t, p in sorted(probs.items(), key=lambda x: -x[1])])
    print(f"{opp} (True: {true_type}): [{prob_str}]")
print("="*90)

SETUP: Bayesian agent plays against 3 opponents whose types it doesn't know
True types: Coop, Grim, Defector

BAYESIAN PRISONER'S DILEMMA
The Bayesian agent doesn't know opponent types - it must infer from observed actions

ROUND 1
------------------------------------------------------------------------------------------
Bayesian's beliefs about opponent types:
  Player-A: [Coop: 33.33%, Defector: 33.33%, Grim: 33.33%]
  Player-B: [Coop: 33.33%, Defector: 33.33%, Grim: 33.33%]
  Player-C: [Coop: 33.33%, Defector: 33.33%, Grim: 33.33%]
Expected payoffs: Cooperate=-7.00, Defect=-3.00
Actions: Bayesian: D  Player-A: C  Player-B: C  Player-C: D
Payoffs: Bayesian: -3  Player-A: -11  Player-B: -11  Player-C: -3

ROUND 2
------------------------------------------------------------------------------------------
Bayesian's beliefs about opponent types:
  Player-A: [Coop: 94.34%, Defector: 0.94%, Grim: 4.72%]
  Player-B: [Coop: 94.34%, Defector: 0.94%, Grim: 4.72%]
  Player-C: [Coop: 0.51%, Defe